Uso de Gradio para crear la interfaz del histograma.

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import glob
import gradio as gr
import seaborn as sns
import matplotlib.pyplot as plt

# Lectura de archivos CSV
path = "samples/"
all_files = glob.glob(path + "/*.csv")
df_list = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_list.append(df)

# Unión de todos los archivos en un solo dataframe y reemplazo de valores faltantes con "UNK".
df_concat = pd.concat(df_list, axis=0, ignore_index=True)
df_concat["Tipo de alerta"] = df_concat["Tipo de alerta"].fillna("UNK")


# Función para calcular estadísticas y visualizar el histograma
def ver_histograma_stats(codigo, tipo_metrica, tipo_alerta):
    # Filtración de los valores específicos
    filtered_df = df_concat[
        (df_concat["Codigo"] == codigo)
        & (df_concat["Clasificacion de metrica"] == tipo_metrica)
        & (df_concat["Tipo de alerta"] == tipo_alerta)
    ]

    # Cálculo de estadísticas básicas
    stats = {
        "Media": filtered_df["Valor"].mean(),
        "Mediana": filtered_df["Valor"].median(),
        "Desviación estándar": filtered_df["Valor"].std(),
        "Mínimo": filtered_df["Valor"].min(),
        "Máximo": filtered_df["Valor"].max(),
        "Cantidad de datos": len(filtered_df),
    }

    # Colocando las stats calculadas de forma legible para la interfaz gráfica
    stats_text = "\n".join([f"{key}: {value:.2f}" for key, value in stats.items()])

    # Creando la nueva figura con dimensiones específicas
    plt.figure(figsize=(10, 6))

    # Creación del histograma con Seaborn
    sns.histplot(filtered_df["Valor"], kde=True, bins=30, color="skyblue")

    # Personalizando la figura creada plt
    plt.title(f"Histograma para {codigo}, {tipo_metrica}, {tipo_alerta}")
    plt.xlabel("Valor")
    plt.ylabel("Frecuencia")
    plt.grid(True)

    return plt, stats_text


# Creación de las entradas como dropdowns
codigo_dropdown = gr.Dropdown(list(df_concat["Codigo"].unique()), label="Código")
clasificacion_metrica_dropdown = gr.Dropdown(
    list(df_concat["Clasificacion de metrica"].unique()),
    label="Clasificacion de métrica",
)
tipo_alerta_dropdown = gr.Dropdown(
    list(df_concat["Tipo de alerta"].unique()), label="Tipo de alerta"
)

# Creación de la interfaz con Gradio
demo = gr.Interface(
    fn=ver_histograma_stats,
    inputs=[codigo_dropdown, clasificacion_metrica_dropdown, tipo_alerta_dropdown],
    outputs=["plot", "text"],
    title="Visualizar histograma y estadísticas",
    description="Seleccione un código, una clasificación de métrica, y un tipo de alerta para visualizar un histograma de los valores y ver estadísticas clave.",
)

# Desplegando la interfaz
demo.launch()

In [6]:
# Deteniendo la demostración
demo.close()

Closing server running on port: 7861
